In [6]:
import os
import pandas as pd
import mne  #https://mne.tools/stable/index.html
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter
#video1[0]
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import pywt
import brainconn as bc
'''
people data load
'''

peoplelist = ['02wxw','03sf','04cmm', '06zhy','08zqh', '09hsq','10mqk','11cx','12wlz','13cxt',
              '14zyr','15yyw','17sh','19hpy','20wrq','22wf','23wh','24hc','25gyz']


for people in peoplelist:
    st = r"..\EEG\ " + people +'_epoched.set'
    st = st.replace(" " , "")
    epochs=mne.io.read_epochs_eeglab(st).apply_baseline(baseline=(-1,0))
    epochs=mne.io.read_epochs_eeglab(st).set_eeg_reference('average').apply_baseline(baseline=(-0.5,0))
    globals()['subject_{}_EEG_data'.format(people)] = epochs.crop(0,3.999).get_data()

Positive = []
Neutral = []
Negative = []
positivevalence = []
neutralvalence = []
negativevalence = []

all_valence_data = pd.read_csv(r'..\valence\valence.txt',delimiter='\t')
for people in peoplelist:
    globals()['subject_{}_valence_data'.format(people)] = all_valence_data[all_valence_data['subject_id'] == people]['valence']
    globals()['subject_{}_valence_data'.format(people)] = globals()['subject_{}_valence_data'.format(people)].reset_index(drop=True)
    positiveind = globals()['subject_{}_valence_data'.format(people)].index[globals()['subject_{}_valence_data'.format(people)]>5].tolist()
    negativeind = globals()['subject_{}_valence_data'.format(people)].index[globals()['subject_{}_valence_data'.format(people)]<5].tolist()
    neutralind = globals()['subject_{}_valence_data'.format(people)].index[globals()['subject_{}_valence_data'.format(people)]==5].tolist()
    print('subject_{}_valence_data'.format(people), "=", len(positiveind+negativeind+neutralind))
    
    for _ in positiveind:
        Positive.append(globals()['subject_{}_EEG_data'.format(people)][_])
        positivevalence.append(globals()['subject_{}_valence_data'.format(people)][_])
    for _ in neutralind:
        Neutral.append(globals()['subject_{}_EEG_data'.format(people)][_])
        neutralvalence.append(globals()['subject_{}_valence_data'.format(people)][_])
    for _ in negativeind:
        Negative.append(globals()['subject_{}_EEG_data'.format(people)][_])
        negativevalence.append(globals()['subject_{}_valence_data'.format(people)][_])

ch_names = epochs.info.ch_names

positive = np.array(Positive)
negative = np.array(Negative)
neutral = np.array(Neutral)

print(positive.shape)
print(negative.shape)
print(neutral.shape)

print("Positive length is ",positive.shape[0])
print("Negative length is ",negative.shape[0])
print("Neutral length is ", neutral.shape[0])
print("Total length is ",negative.shape[0]+positive.shape[0]+neutral.shape[0])

window = signal.windows.hann(1024)

positive_filter_EEG = np.array(np.array(Positive))
negative_filter_EEG = np.array(np.array(Negative))
neutral_filter_EEG = np.array(np.array(Neutral))
import time
from datetime import datetime

current_time = datetime.now()
print(current_time)

start = time.time()

'''
delta = [0, 2] #index값 0,1.95,3.9 
theta = [3, 4] #5.85, 7.81
alpha = [5, 6] #9.76,11.71
beta = [7, 15] #13.67, 29.29
gamma = [16, 25] #31.25 48.82
high gamma = [52, 83] # 50.78125 80.078125

'''
bandfreq = [[0,5,'delta'],[5,9,'theta'],[9,15,'alpha'],[15,32,'beta'],[32,52,'gamma'],[52,83,'highgamma']]
Valence = [['positive',708],['negative',696],['neutral',639]]


for val in Valence: #positive, negative, neutral
    for band in bandfreq: #delta, theta, alpha, beta, gamma
        globals()['{}_{}_coh_matrixs'.format(val[0], band[2])]=[]
        for i in range(val[1]):
            globals()['{}_{}_cohmat'.format(val[0],band[2])] = [] # 59*59
            for c in range(chan_len):
                globals()['{}_coh'.format(val[0])] =[] # 1*59
                for j in range(chan_len):
                    _, coh = signal.coherence(globals()['{}_filter_EEG'.format(val[0])][i][c],globals()['{}_filter_EEG'.format(val[0])][i][j],fs =1000,window=window)
                    cohb = np.mean(coh[band[0]:band[1]])
                    globals()['{}_coh'.format(val[0])].append(cohb)
                globals()['{}_{}_cohmat'.format(val[0],band[2])].append(globals()['{}_coh'.format(val[0])])
            globals()['{}_{}_coh_matrixs'.format(val[0], band[2])].append(globals()['{}_{}_cohmat'.format(val[0],band[2])])
        print('{}_{}_completed!'.format(val[0], band[2]))

        
        
print("time :", ((time.time() - start)/60)/60 ,"hours")
current_time = datetime.now()
print(current_time)

'''
소수점 버림
'''
for val in Valence: #positive, negative, neutral
    for band in bandfreq: #delta, theta, alpha, beta, gamma
        globals()['{}_{}_coh_matrixs'.format(val[0], band[2])] = np.round(globals()['{}_{}_coh_matrixs'.format(val[0], band[2])],3) 
        np.fill_diagonal(globals()['{}_{}_coh_matrixs'.format(val[0], band[2])][i] ,0)
        
for val in Valence: #positive, negative, neutral
    for band in bandfreq: #delta, theta, alpha, beta, gamma
         globals()['{}_{}_EEG'.format(val[0], band[2])] = np.array(globals()['{}_{}_coh_matrixs'.format(val[0], band[2])])

for fr in bandfreq:
    for num in range(positive_delta_EEG.shape[0]): # Positive Length:
        np.fill_diagonal(globals()['positive_{}_EEG'.format(fr[2])][num],0)
    for num in range(negative_delta_EEG.shape[0]): # Negative Length:
        np.fill_diagonal(globals()['negative_{}_EEG'.format(fr[2])][num],0)
    for num in range(neutral_delta_EEG.shape[0]): # Neutral Length:
        np.fill_diagonal(globals()['neutral_{}_EEG'.format(fr[2])][num],0)
        
'''
Plotting subject mean coherence matrix
'''
bandfreq = [[0,5,'delta'],[5,9,'theta'],[9,15,'alpha'],[15,32,'beta'],[32,52,'gamma'],[52,83,'highgamma']]
Valence = [['positive',708],['negative',696],['neutral',639]]
for val in Valence:
    for band in bandfreq:
        globals()['{}_{}_mean_matrix'.format(val[0], band[2])] = np.mean(np.array(globals()['{}_{}_coh_matrixs'.format(val[0], band[2])]),axis=0)
import seaborn as sns
fig, axes = plt.subplots(3,6,figsize=(20,8))

for i, val in enumerate(Valence):
    for q, band in enumerate(bandfreq):
        a= pd.DataFrame(globals()['{}_{}_mean_matrix'.format(val[0], band[2])], columns = ch_names, index = ch_names)
        sns.heatmap(a,cmap='jet',ax = axes[i][q],vmin=0.2,vmax=1)
        if i == 0:
            axes[i][q].set_title(band[2])
        if q ==0:
            axes[i][q].set_ylabel(val[0])
plt.savefig("Fig/Mean coherence map", dpi = 600)


'''
positive_delta_EEG
example.
we use cij 
cij , _ = bc.utils.visualization.backbone_wu(positive_alpha_EEG[13],3)
'''
bandfreq = [[0,5,'delta'],[5,9,'theta'],[9,15,'alpha'],[15,32,'beta'],[32,52,'gamma'],[52,83,'highgamma']]
Valence = [['positive',708],['negative',696],['neutral',639]]
current_time = datetime.now()
print(current_time)
start = time.time()

for fr in bandfreq:
    for num in range(globals()['positive_{}_EEG'.format(fr[2])].shape[0]): # Positive Length:
        _, globals()['Positive_{}_{}_corrcoef_Matrix_binarize'.format(fr[2], num)] = bc.utils.visualization.backbone_wu(globals()['positive_{}_EEG'.format(fr[2])][num], 5)
    for num in range(globals()['negative_{}_EEG'.format(fr[2])].shape[0]): # Negative Length:
        _, globals()['Negative_{}_{}_corrcoef_Matrix_binarize'.format(fr[2],num)]= bc.utils.visualization.backbone_wu(globals()['negative_{}_EEG'.format(fr[2])][num], 5)
    for num in range(globals()['neutral_{}_EEG'.format(fr[2])].shape[0]): # Neutral Length:
        _, globals()['Neutral_{}_{}_corrcoef_Matrix_binarize'.format(fr[2],num)]= bc.utils.visualization.backbone_wu(globals()['neutral_{}_EEG'.format(fr[2])][num], 5)
print("time :", ((time.time() - start)/60)/60 ,"hours")
current_time = datetime.now()
print(current_time)


'''
mean map 그려보기
'''
bandfreq = [[0,5,'delta'],[5,9,'theta'],[9,15,'alpha'],[15,32,'beta'],[32,52,'gamma'],[52,83,'highgamma']]
Valence = [['positive',708],['negative',696],['neutral',639]]
for val in Valence:
    for band in bandfreq:
        globals()['{}_{}_mean_mst'.format(val[0], band[2])] =[]

for fr in bandfreq:
    for num in range(globals()['positive_{}_EEG'.format(fr[2])].shape[0]): # Positive Length:
        globals()['positive_{}_mean_mst'.format(fr[2])].append(globals()['Positive_{}_{}_corrcoef_Matrix_binarize'.format(fr[2], num)])
    for num in range(globals()['negative_{}_EEG'.format(fr[2])].shape[0]): # Negative Length:
        globals()['negative_{}_mean_mst'.format(fr[2])].append(globals()['Negative_{}_{}_corrcoef_Matrix_binarize'.format(fr[2],num)]) 
    for num in range(globals()['neutral_{}_EEG'.format(fr[2])].shape[0]): # Neutral Length:
        globals()['neutral_{}_mean_mst'.format(fr[2])].append(globals()['Neutral_{}_{}_corrcoef_Matrix_binarize'.format(fr[2],num)])

for val in Valence:
    for band in bandfreq:
        globals()['{}_{}_mean_mst'.format(val[0], band[2])] = np.mean(np.array(globals()['{}_{}_mean_mst'.format(val[0], band[2])]),axis=0)

        
for fr in bandfreq:
    for num in range(globals()['positive_{}_EEG'.format(fr[2])].shape[0]): # Positive Length:
        globals()['Positive_{}_{}_corrcoef_Matrix_binarize'.format(fr[2], num)][np.where(globals()['Positive_{}_{}_corrcoef_Matrix_binarize'.format(fr[2], num)]>0)]=1
    for num in range(globals()['negative_{}_EEG'.format(fr[2])].shape[0]): # Negative Length:
        globals()['Negative_{}_{}_corrcoef_Matrix_binarize'.format(fr[2], num)][np.where(globals()['Negative_{}_{}_corrcoef_Matrix_binarize'.format(fr[2], num)]>0)]=1
    for num in range(globals()['neutral_{}_EEG'.format(fr[2])].shape[0]): # Neutral Length:
        globals()['Neutral_{}_{}_corrcoef_Matrix_binarize'.format(fr[2], num)][np.where(globals()['Neutral_{}_{}_corrcoef_Matrix_binarize'.format(fr[2], num)]>0)]=1

fig, axes = plt.subplots(3,6,figsize=(20,8))
for i, val in enumerate(Valence):
    for q, band in enumerate(bandfreq):
        a= pd.DataFrame(globals()['{}_{}_mean_mst'.format(val[0], band[2])], columns = ch_names, index = ch_names)
        
        sns.heatmap(a,cmap='jet',ax = axes[i][q])
        if i == 0:
            axes[i][q].set_title(band[2])
        if q ==0:
            axes[i][q].set_ylabel(val[0])

            
            
fig, axes = plt.subplots(1,6,figsize=(20,3))
color_map = plt.cm.get_cmap('gray')
reversed_color_map = color_map.reversed()
vale = ['Positive','Neutral', 'Negative']
band = ['delta','theta','alpha', 'beta','gamma','highgamma']
for val in vale:
    for bd in band:
        globals()["{}_{}".format(val, bd)] = pd.DataFrame(globals()["{}_{}_0_corrcoef_Matrix_binarize".format(val, bd)],columns = ch_names, index = ch_names)

for i, bd in enumerate(band):
    sns.heatmap(globals()["Positive_{}".format(bd)],cmap=reversed_color_map,ax = axes[i])
    
#################### Graph_measure


'''
CP
'''
'''
Optain distance matrix
'''
vale = ['Positive','Neutral', 'Negative']
band = ['delta','theta','alpha', 'beta','gamma','highgamma']
leng = [708,639,696]

for val in vale:
    for q in band:
        globals()['C_P_{}_{}'.format(val, q)] = []

_, Dim = bc.distance.breadthdist(Positive_delta_1_corrcoef_Matrix_binarize)
bc.distance.charpath(Dim)[0]
for ind, a in enumerate(leng):
    for i in range(a):
        for q in band:
            globals()['C_P_{}_{}'.format(vale[ind],q)].append(bc.distance.charpath(bc.distance.distance_bin(globals()['{}_{}_{}_corrcoef_Matrix_binarize'.format(vale[ind],q, i)]), include_infinite=False)[0])
            

def set_box_color(bp, color):
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color=color)

raw3_data ={
    'delta':C_P_Positive_delta,
    'theta':C_P_Positive_theta,
    'alpha':C_P_Positive_alpha,
    'beta':C_P_Positive_beta,
    'gamma':C_P_Positive_gamma,
    'highgamma':C_P_Positive_highgamma
}


raw4_data ={
    'delta':C_P_Negative_delta,
    'theta':C_P_Negative_theta,
    'alpha':C_P_Negative_alpha,
    'beta':C_P_Negative_beta,
    'gamma':C_P_Negative_gamma,
    'highgamma':C_P_Negative_highgamma
}


raw5_data ={
    'delta':C_P_Neutral_delta,
    'theta':C_P_Neutral_theta,
    'alpha':C_P_Neutral_alpha,
    'beta':C_P_Neutral_beta,
    'gamma':C_P_Neutral_gamma,
    'highgamma':C_P_Neutral_highgamma
}


positive_C_P_df = pd.DataFrame(raw3_data)
negative_C_P_df = pd.DataFrame(raw4_data)
neutral_C_P_df = pd.DataFrame(raw5_data)

plt.figure(figsize=(12,8))
ticks=['delta','theta', 'alpha','beta','gamma','highgamma']


range(0,len(ticks))
bpl = positive_C_P_df.boxplot(column=['delta',
                                    'theta', 'alpha','beta', 'gamma','highgamma'],grid=False,color='red',
                                  positions=np.array(range(6))*2.0-0.3,widths=0.2)

bpr = negative_C_P_df.boxplot(column=['delta',
                                    'theta', 'alpha','beta', 'gamma','highgamma'],grid=False ,color='blue',
                                  positions=np.array(range(6))*2.0,widths=0.2)
bps = neutral_C_P_df.boxplot(column=['delta',
                                    'theta', 'alpha','beta', 'gamma','highgamma'],grid=False ,color='black',
                                  positions=np.array(range(6))*2.0+0.3,widths=0.2)

plt.xticks(range(0, len(ticks) * 2, 2), ticks)
plt.plot([], c='red', label='Positive')
plt.plot([], c='blue', label='Negative')
plt.plot([], c='black', label='Neutral')
plt.legend(edgecolor='white')
plt.gca().spines[['top', 'right']].set_visible(False)
plt.tick_params(axis='x', direction='in', top=False)
plt.tick_params(axis='y', direction='in', top=False)
plt.grid(True, axis='y')
#plt.ylim(1.6,2.6)
plt.title("Characteristic path length",fontsize=20)
plt.xlabel("Frequency Band",fontsize=13)
plt.ylabel("Characteristic path length",fontsize=13)
plt.savefig("Fig/Characteristic path length.svg",bbox_inches='tight')
'''

violin plot data 정리

'''
positive_val = []
negative_val = []
neutral_val = []

val = ['positive','negative','neutral']
band = ['delta','theta','alpha','beta','gamma','highgamma']

for v in val:
    for b in band:
        globals()['{}_{}'.format(v,b)] =[]


for i in range(708):
    positive_val.append("Positive")
    for b in band:
        globals()['positive_{}'.format(b)].append(b)
        
for i in range(696):
    negative_val.append("Negative")
    for b in band:
        globals()['negative_{}'.format(b)].append(b)

for i in range(639):
    neutral_val.append("Neutral")
    for b in band:
        globals()['neutral_{}'.format(b)].append(b)
a =[]
for v in val:
    for b in band:
        globals()['{}_{}_C_P_df'.format(v,b)] = pd.DataFrame(globals()['{}_C_P_df'.format(v)][b])
        globals()['{}_{}_C_P_df'.format(v,b)].columns = ['value']
        globals()['{}_{}_C_P_df'.format(v,b)]['band'] = globals()['{}_{}'.format(v, b)]
        globals()['{}_{}_C_P_df'.format(v,b)]['valence'] = globals()['{}_val'.format(v)]
        a.append(globals()['{}_{}_C_P_df'.format(v,b)])
C_P_df = pd.concat(a)


def set_box_color(bp, color):
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color=color)

raw6_data ={
    'delta':G_E_Positive_delta,
    'theta':G_E_Positive_theta,
    'alpha':G_E_Positive_alpha,
    'beta':G_E_Positive_beta,
    'gamma':G_E_Positive_gamma,
    'highgamma':G_E_Positive_highgamma
}


raw7_data ={
    'delta':G_E_Negative_delta,
    'theta':G_E_Negative_theta,
    'alpha':G_E_Negative_alpha,
    'beta':G_E_Negative_beta,
    'gamma':G_E_Negative_gamma,
    'highgamma':G_E_Negative_highgamma
}


raw8_data ={
    'delta':G_E_Neutral_delta,
    'theta':G_E_Neutral_theta,
    'alpha':G_E_Neutral_alpha,
    'beta':G_E_Neutral_beta,
    'gamma':G_E_Neutral_gamma,
    'highgamma':G_E_Neutral_highgamma
}





positive_G_E_df = pd.DataFrame(raw6_data)
negative_G_E_df = pd.DataFrame(raw7_data)
neutral_G_E_df = pd.DataFrame(raw8_data)


plt.figure(figsize=(12,8))
ticks=['delta','theta', 'alpha','beta','gamma']


range(0,len(ticks))
bpl = positive_G_E_df.boxplot(column=['delta',
                                    'theta', 'alpha','beta', 'gamma','highgamma'],grid=False,color='red',
                                  positions=np.array(range(6))*2.0-0.3,widths=0.2)

bpr = negative_G_E_df.boxplot(column=['delta',
                                    'theta', 'alpha','beta', 'gamma','highgamma'],grid=False ,color='blue',
                                  positions=np.array(range(6))*2.0+0.0,widths=0.2)
bps = neutral_G_E_df.boxplot(column=['delta',
                                    'theta', 'alpha','beta', 'gamma','highgamma'],grid=False ,color='black',
                                  positions=np.array(range(6))*2.0+0.3,widths=0.2)


plt.xticks(range(0, len(ticks) * 2, 2), ticks)
plt.plot([], c='red', label='Positive')
plt.plot([], c='blue', label='Negative')
plt.plot([], c='black', label='Neutral')

plt.legend(edgecolor='white')
plt.gca().spines[['top', 'right']].set_visible(False)
plt.tick_params(axis='x', direction='in', top=False)
plt.tick_params(axis='y', direction='in', top=False)
plt.grid(True, axis='y')
#plt.ylim(0.46,0.64)
plt.title("Global Effeciency",fontsize=20)
plt.xlabel("Frequency Band",fontsize=13)
plt.ylabel("Global Effeciency",fontsize=13)

plt.savefig("Fig/Global Effeciency.svg",bbox_inches='tight')
'''

violin plot data 정리

'''
positive_val = []
negative_val = []
neutral_val = []

val = ['positive','negative','neutral']
band = ['delta','theta','alpha','beta','gamma','highgamma']

for v in val:
    for b in band:
        globals()['{}_{}'.format(v,b)] =[]


for i in range(708):
    positive_val.append("Positive")
    for b in band:
        globals()['positive_{}'.format(b)].append(b)
        
for i in range(696):
    negative_val.append("Negative")
    for b in band:
        globals()['negative_{}'.format(b)].append(b)

for i in range(639):
    neutral_val.append("Neutral")
    for b in band:
        globals()['neutral_{}'.format(b)].append(b)
a =[]
for v in val:
    for b in band:
        globals()['{}_{}_G_E_df'.format(v,b)] = pd.DataFrame(globals()['{}_G_E_df'.format(v)][b])
        globals()['{}_{}_G_E_df'.format(v,b)].columns = ['value']
        globals()['{}_{}_G_E_df'.format(v,b)]['band'] = globals()['{}_{}'.format(v, b)]
        globals()['{}_{}_G_E_df'.format(v,b)]['valence'] = globals()['{}_val'.format(v)]
        a.append(globals()['{}_{}_G_E_df'.format(v,b)])

G_E_df = pd.concat(a)


'''
ANOVA TEST
'''

bands = ['delta','theta','alpha', 'beta','gamma','highgamma']

print("글로벌 지수에 대한 통계분석 수행 결과: ")
print("Characteristic path length")
for band in bands:
    print(band)
    print(pg.welch_anova(data = C_P_df[C_P_df['band']==band], dv = 'value' ,between ='valence'))
    
#plt.kdeplot(C_P_df[C_P_df['band']=='theta'])

print("Global Effeciency")
for band in bands:
    print(band)
    print(pg.welch_anova(data = G_E_df[G_E_df['band']==band], dv = 'value' ,between ='valence'))

vale = ['Positive','Neutral', 'Negative']
band = ['delta','theta','alpha', 'beta','gamma','highgamma']
leng = [708,639,696]


for val in vale:
    for bd in band:
        globals()['clu_{}_{}'.format(val, bd)]=[]

# calculate Clustering Coefficient
for i in range(leng[0]):
    for q in band:
        globals()['clu_Positive_{}'.format(q)].append(bc.clustering.clustering_coef_wu(globals()['Positive_{}_{}_corrcoef_Matrix_binarize'.format(q, i)]))

for i in range(leng[1]):
    for q in band:
        globals()['clu_Neutral_{}'.format(q)].append(bc.clustering.clustering_coef_wu(globals()['Neutral_{}_{}_corrcoef_Matrix_binarize'.format(q, i)]))

for i in range(leng[2]):
    for q in band:
        globals()['clu_Negative_{}'.format(q)].append(bc.clustering.clustering_coef_wu(globals()['Negative_{}_{}_corrcoef_Matrix_binarize'.format(q, i)]))

vale = ['Positive','Neutral', 'Negative']
band = ['delta','theta','alpha', 'beta','gamma','highgamma']
leng = [708,639,696]

for val in vale:
    for bd in band:
        globals()['clu_{}_{}_df'.format(val, bd)]=pd.DataFrame(globals()['clu_{}_{}'.format(val, bd)],columns= ch_names)
        
band = ['delta','theta','alpha', 'beta','gamma','highgamma']
val =['Positive','Neutral','Negative']
Positive_val = []
Negative_val = []
Neutral_val = []
leng = [708, 639, 696] # positive, neutral, negative'

for v in val:
    for b in band:
        globals()['{}_{}'.format(v,b)] =[]


for b in band:    
    for i in range(708):
        globals()['Positive_{}'.format(b)].append(b)
    
for b in band:    
    for i in range(639):
        globals()['Neutral_{}'.format(b)].append(b)

for b in band:    
    for i in range(696):
        globals()['Negative_{}'.format(b)].append(b)
  
        
###################################################

for i in range(708):
    Positive_val.append('Positive')
    
for i in range(639):
    Neutral_val.append('Neutral')

for i in range(696):
    Negative_val.append('Negative')
        
############################################################        


for va in val:
    for ch in ch_names:
        globals()['{}_{}'.format(va, ch)] = []
    
for i in range(leng[0]): #positive leng
    for ch in ch_names: 
        globals()['{}_{}'.format(val[0],ch)].append(ch)

for i in range(leng[1]): #negative leng
    for ch in ch_names: 
        globals()['{}_{}'.format(val[1],ch)].append(ch)
        
for i in range(leng[2]): #neutral leng
    for ch in ch_names: 
        globals()['{}_{}'.format(val[2],ch)].append(ch)

        
#globals()['clu_{}_{}_df'.format(v,b)][ch]


a=[]
for v in val:
    for b in band:    
        for ch in ch_names:
            my_df = pd.DataFrame(globals()['clu_{}_{}_df'.format(v,b)][ch])
            my_df.columns =['value']
            my_df['ch'] = globals()['{}_{}'.format(v,ch)]
            my_df['band'] = globals()['{}_{}'.format(v,b)]
            my_df['valence']=globals()['{}_val'.format(v)]
            a.append(my_df)
            
C_C = pd.concat(a)

import mne
a = mne.channels.make_standard_montage("standard_1020")
std_loc = mne.channels.DigMontage.get_positions(a)
std_loc = std_loc['ch_pos']
locc = {}
for ch in ch_names:
    locc[ch]= std_loc[ch]

'''
전극위치구성
'''
custom = mne.channels.make_dig_montage(ch_pos = locc)
custom_info = mne.create_info(ch_names=custom.ch_names, sfreq=1.,
                            ch_types='eeg')
data = np.random.normal(size=(59, 1)) * 1e-6
custom_evoked = mne.EvokedArray(data, custom_info)
custom_evoked.set_montage(custom)
vale = ['Positive','Neutral', 'Negative']
band = ['delta','theta','alpha', 'beta','gamma','highgamma']
for val in vale:
    for bd in band:
        globals()['mean_clu_{}_{}_df'.format(val, bd)]= np.array(globals()['clu_{}_{}_df'.format(val, bd)].mean())
        
vmin=100000
vmax = 0
for i, val in enumerate(vale):
    for q, ba in enumerate(band):
        vmin = min(vmin, min(globals()['mean_clu_{}_{}_df'.format(val, ba)]))
        vmax = max(vmax, max(globals()['mean_clu_{}_{}_df'.format(val, ba)]))
import numpy as np
import mne
fig, ax = plt.subplots(nrows=3,ncols=6, figsize=(22, 10), gridspec_kw={'width_ratios': [12, 12,12,12,12,15.5]},
                       sharex=True, sharey=True)
np.random.seed(1)
for i, val in enumerate(vale):
    for q, ba in enumerate(band):
        s = mne.viz.plot_topomap(globals()['mean_clu_{}_{}_df'.format(val, ba)], custom_evoked.info, axes=ax[i][q],show=False,cmap ='jet',names = ch_names, 
                                 sphere = 0.108,vmin=vmin, vmax=vmax)
        if i == 0:
            ax[i][q].set_title(ba,fontweight='bold')
        if q ==0:
            ax[i][q].set_ylabel(val,fontweight='bold',fontsize=16)  
        if q == 5:
            mynorm = plt.Normalize(vmin=vmin, vmax=vmax)
            sm = plt.cm.ScalarMappable(cmap='jet', norm=mynorm)
            sm_= plt.colorbar(sm, ax = ax[i][q])
plt.suptitle("Fig/Clustering coefficient",fontweight='bold', fontsize=20, y = 1.02)
'''
Example Betweenness Centrality
return N*1
'''

vale = ['Positive','Neutral', 'Negative']
band = ['delta','theta','alpha', 'beta','gamma','highgamma']
leng = [708,639,696]

for val in vale:
    for q in band:
        globals()['B_C_{}_{}'.format(val, q)] = []

# calculate Betweenness Centrality
for ind, a in enumerate(leng):
    for i in range(a):
        for q in band:
            globals()['B_C_{}_{}'.format(vale[ind],q)].append(bc.centrality.betweenness_bin(globals()['{}_{}_{}_corrcoef_Matrix_binarize'.format(vale[ind],q, i)]))
for val in vale:
    for q in band:
        globals()['B_C_{}_{}_df'.format(val, q)] = pd.DataFrame(globals()['B_C_{}_{}'.format(val, q)],columns= ch_names)
val =['Positive','Neutral','Negative']
Positive_val = []
Negative_val = []
Neutral_val = []
leng = [708, 639, 696] # positive, neutral, negative'

for v in val:
    for b in band:
        globals()['{}_{}'.format(v,b)] =[]


for b in band:    
    for i in range(708):
        globals()['Positive_{}'.format(b)].append(b)
    
for b in band:    
    for i in range(639):
        globals()['Neutral_{}'.format(b)].append(b)

for b in band:    
    for i in range(696):
        globals()['Negative_{}'.format(b)].append(b)
  
        
###################################################

for i in range(708):
    Positive_val.append('Positive')
    
for i in range(639):
    Neutral_val.append('Neutral')

for i in range(696):
    Negative_val.append('Negative')
        
############################################################        


for va in val:
    for ch in ch_names:
        globals()['{}_{}'.format(va, ch)] = []
    
for i in range(leng[0]): #positive leng
    for ch in ch_names: 
        globals()['{}_{}'.format(val[0],ch)].append(ch)

for i in range(leng[1]): #negative leng
    for ch in ch_names: 
        globals()['{}_{}'.format(val[1],ch)].append(ch)
        
for i in range(leng[2]): #neutral leng
    for ch in ch_names: 
        globals()['{}_{}'.format(val[2],ch)].append(ch)

        
#globals()['clu_{}_{}_df'.format(v,b)][ch]


a=[]
for v in val:
    for b in band:    
        for ch in ch_names:
            my_df = pd.DataFrame(globals()['B_C_{}_{}_df'.format(v, b)][ch])
            my_df.columns =['value']
            my_df['ch'] = globals()['{}_{}'.format(v,ch)]
            my_df['band'] = globals()['{}_{}'.format(v,b)]
            my_df['valence']=globals()['{}_val'.format(v)]
            a.append(my_df)
            
B_C = pd.concat(a)
B_C
vale = ['Positive','Neutral', 'Negative']
band = ['delta','theta','alpha', 'beta','gamma','highgamma']
for val in vale:
    for bd in band:
        globals()['mean_B_C_{}_{}_df'.format(val, bd)]= np.array(globals()['B_C_{}_{}_df'.format(val, bd)].mean())
vmin=100000
vmax = 0
print(band)
for i, val in enumerate(vale):
    for q, ba in enumerate(band):
        vmin = min(vmin, min(globals()['mean_B_C_{}_{}_df'.format(val, ba)]))
        vmax = max(vmax, max(globals()['mean_B_C_{}_{}_df'.format(val, ba)]))    
import numpy as np
import mne

fig, ax = plt.subplots(nrows=3,ncols=6, figsize=(20, 12), sharex=True, sharey=True
                      ,gridspec_kw={'width_ratios': [12, 12,12,12,12,15.5]})
#np.random.seed(1)

for i, val in enumerate(vale):
    for q, ba in enumerate(band):
        mne.viz.plot_topomap(globals()['mean_B_C_{}_{}_df'.format(val, ba)], custom_evoked.info, axes=ax[i][q],show=False,cmap ='jet', 
                             sphere = 0.108,names = ch_names,show_names =False,vmin=vmin, vmax=vmax)
        if i == 0:
            ax[i][q].set_title(ba,fontweight='bold')
        if q ==0:
            ax[i][q].set_ylabel(val,fontweight='bold',fontsize=16)
        if q == 5:
            mynorm = plt.Normalize(vmin=vmin, vmax=vmax)
            sm = plt.cm.ScalarMappable(cmap='jet', norm=mynorm)
            sm_= plt.colorbar(sm, ax = ax[i][q])            
plt.suptitle("Betweenness Centrality",fontweight='bold', fontsize=20,y = 1.02)
plt.savefig("Fig/Betweenness_Centrality_topomap",dpi  = 600)
'''
통계분석
CC

'''
import pingouin as pg

band = ['delta','theta','alpha', 'beta','gamma','highgamma']
cc_anova = {'delta':{},'theta':{},'alpha':{},'beta':{},'gamma':{},'highgamma':{}} #Negative - Neutral {band: {ch : p-value}}
cc_Ttest = {'delta':{},'theta':{},'alpha':{},'beta':{},'gamma':{},'highgamma':{}}

for b in band:
    for ch in ch_names:
        #ANOVA 적용
        a = pg.welch_anova(data = C_C.loc[(C_C['band']==b) & (C_C['ch']==ch)], dv = 'value' ,between ='valence')
        if a['p-unc'][0] < 0.05:
            pw = pg.pairwise_gameshowell(data = C_C.loc[(C_C['band']==b) & (C_C['ch']==ch)], dv = 'value' ,between ='valence')
            pwtest = {'A': pw['pval'][0],'B': pw['pval'][1],'C': pw['pval'][2]}
            cc_anova[b][ch] = pwtest
            ccT = {}
            
            if pw['pval'][0] < 0.05:
                ccT['A'] = pw['T'][0]
            else:
                ccT['A'] = 0
            
            if pw['pval'][1] < 0.05:
                ccT['B'] = pw['T'][1]
            else:
                ccT['B'] = 0
            
            if pw['pval'][2] < 0.05:
                ccT['C'] = pw['T'][2]
            else:
                ccT['C'] = 0
                
            cc_Ttest[b][ch] = ccT
            
        elif a['p-unc'][0] > 0.05:
            ttest = {'A':0,'B':0,'C':0}
            cc_Ttest[b][ch] = ttest
'''

BC

'''


band = ['delta','theta','alpha', 'beta','gamma','highgamma']
bc_anova = {'delta':{},'theta':{},'alpha':{},'beta':{},'gamma':{},'highgamma':{}} #Negative - Neutral {band: {ch : p-value}}
bc_Ttest = {'delta':{},'theta':{},'alpha':{},'beta':{},'gamma':{},'highgamma':{}}


for b in band:
    for ch in ch_names:
        #ANOVA 적용
        a = pg.welch_anova(data = B_C.loc[(B_C['band']==b) & (B_C['ch']==ch)], dv = 'value' ,between ='valence')
        if a['p-unc'][0] < 0.05:
            pw = pg.pairwise_gameshowell(data = B_C.loc[(B_C['band']==b) & (B_C['ch']==ch)], dv = 'value' ,between ='valence')
            pwtest = {'A': pw['pval'][0],'B': pw['pval'][1],'C': pw['pval'][2]}
            bc_anova[b][ch] = pwtest
            ccT = {}
            
            if pw['pval'][0] < 0.05:
                ccT['A'] = pw['T'][0]
            else:
                ccT['A'] = 0
            
            if pw['pval'][1] < 0.05:
                ccT['B'] = pw['T'][1]
            else:
                ccT['B'] = 0
            
            if pw['pval'][2] < 0.05:
                ccT['C'] = pw['T'][2]
            else:
                ccT['C'] = 0
                
            bc_Ttest[b][ch] = ccT

            
        elif a['p-unc'][0] > 0.05:
            ttest = {'A':0,'B':0,'C':0}
            bc_Ttest[b][ch] = ttest
band = ['delta','theta','alpha', 'beta','gamma', 'highgamma']
sub = 1
plt.figure(figsize=(55, 55))
for i, bd in enumerate(band):
    a = list(cc_anova[bd].keys())
    for q, ch in enumerate(a):
        data = C_C.loc[(C_C['band']==bd) & (C_C['ch']==ch)]

        plt.subplot(7,10,sub)
        
        bpl =pd.DataFrame(data[data['valence']=='Positive']).boxplot(grid=False,color = 'k',widths=0.15,positions=np.array(range(1))*2.0-0.3)
        bps = pd.DataFrame(data[data['valence']=='Neutral']).boxplot(grid=False ,color = 'k',widths=0.15,positions=np.array(range(1))*2.0)
        bpr = pd.DataFrame(data[data['valence']=='Negative']).boxplot(grid=False ,color = 'k',widths=0.15,positions=np.array(range(1))*2.0+0.3)

        plt.xticks([-0.3,0,0.3], ['Pos','Neu',  'Neg'])
        plt.gca().spines[['top', 'right']].set_visible(False)
        plt.tick_params(axis='x', direction='in', top=False)
        plt.tick_params(axis='y', direction='in', top=False)
        count =0
        
        if cc_anova[bd][ch]['A']<0.05:
            count += 1
            ff1(cc_anova[bd][ch]['A'])
            
        if cc_anova[bd][ch]['B']<0.05:
            count += 1 
            ff2(cc_anova[bd][ch]['B'])
            
        if cc_anova[bd][ch]['C']<0.05:
            count += 1
            ff3(cc_anova[bd][ch]['C'])     
            
            
        if cc_anova[bd][ch]['A']>0.05 and cc_anova[bd][ch]['B']>0.05 and cc_anova[bd][ch]['C']>0.05:
            plt.annotate('Gameshowell X', xy=(0.05, 0.85), xycoords='axes fraction')
            
        pos =np.mean(data[data['valence']=='Positive']['value'])
        neg = np.mean(data[data['valence']=='Negative']['value'])
        neu = np.mean(data[data['valence']=='Neutral']['value'])
        
        
        if cc_anova[bd][ch]['A']<0.05:
            if count == 1:
                if neg<neu:
                    plt.annotate('Neg<Neu', xy=(0.05, 0.85), xycoords='axes fraction')
                else:
                    plt.annotate('Neu<Neg', xy=(0.05, 0.85), xycoords='axes fraction')
                    
            if count == 2:
                if neg<neu:
                    plt.annotate('Neg<Neu', xy=(0.05, 0.85), xycoords='axes fraction')
                else:
                    plt.annotate('Neu<Neg', xy=(0.05, 0.85), xycoords='axes fraction')
                    
            elif count == 3:
                if pos<neg<neu:
                     plt.annotate('Pos<Neg<Neu', xy=(0.05, 0.85), xycoords='axes fraction')
                elif pos<neu<neg:
                    plt.annotate('Pos<Neu<Neg', xy=(0.05, 0.85), xycoords='axes fraction')
                elif neg<neu<pos:
                    plt.annotate('Neg<Neu<Pos', xy=(0.05, 0.85), xycoords='axes fraction')
                elif neg<pos<neu:
                    plt.annotate('Neg<Pos<Neu', xy=(0.05, 0.85), xycoords='axes fraction')
                elif neu<pos<neg:
                    plt.annotate('Neu<Pos<Neg', xy=(0.05, 0.85), xycoords='axes fraction')
                elif neu<neg<pos:
                    plt.annotate('Neu<Neg<Pos', xy=(0.05, 0.85), xycoords='axes fraction')
                
                
        if cc_anova[bd][ch]['B']<0.05:
            if count == 1:
                if pos<neg:
                    plt.annotate('Pos<Neg', xy=(0.05, 0.85), xycoords='axes fraction')
                else:
                    plt.annotate('Neg<Pos', xy=(0.05, 0.85), xycoords='axes fraction')
            if count == 2:
                if pos<neg:
                    plt.annotate('Pos<Neg', xy=(0.05, 0.78), xycoords='axes fraction')
                else:
                    plt.annotate('Neg<Pos', xy=(0.05, 0.78), xycoords='axes fraction')
        
        
        if cc_anova[bd][ch]['C']<0.05:
            if count == 1:
                if pos<neu:
                    plt.annotate('Pos<Neu', xy=(0.05, 0.85), xycoords='axes fraction')
                else:
                    plt.annotate('Neu<Pos', xy=(0.05, 0.85), xycoords='axes fraction')
                    
                    
            if count == 2:
                if pos<neu:
                    plt.annotate('Pos<Neu', xy=(0.05, 0.78), xycoords='axes fraction')
                else:
                    plt.annotate('Neu<Pos', xy=(0.05, 0.78), xycoords='axes fraction')
                
        plt.title("CC {} {}".format(bd,ch), pad=20)
        sub+=1

plt.savefig("Fig/COH_CC.jpg", dpi= 600,bbox_inches='tight') 
band = ['delta','theta','alpha', 'beta','gamma','highgamma']
sub = 1
plt.figure(figsize=(65, 65))
for bd in band:
    a = list(bc_anova[bd].keys())
    for ch in a:
        plt.subplot(8,10,sub)
        data = B_C.loc[(B_C['band']==bd) & (B_C['ch']==ch)]
        
        bpl =pd.DataFrame(data[data['valence']=='Positive']).boxplot(grid=False,color = 'k',widths=0.15,positions=np.array(range(1))*2.0-0.3)
        bps = pd.DataFrame(data[data['valence']=='Neutral']).boxplot(grid=False ,color = 'k',widths=0.15,positions=np.array(range(1))*2.0)
        bpr = pd.DataFrame(data[data['valence']=='Negative']).boxplot(grid=False ,color = 'k',widths=0.15,positions=np.array(range(1))*2.0+0.3)

        plt.xticks([-0.3,0,0.3], ['Pos','Neu',  'Neg'])
        plt.gca().spines[['top', 'right']].set_visible(False)
        plt.tick_params(axis='x', direction='in', top=False)
        plt.tick_params(axis='y', direction='in', top=False)
        count =0
        
        if bc_anova[bd][ch]['A']<0.05:
            count += 1
            ff1(bc_anova[bd][ch]['A'])
            
        if bc_anova[bd][ch]['B']<0.05:
            count += 1
            ff2(bc_anova[bd][ch]['B'])
            
        if bc_anova[bd][ch]['C']<0.05:
            count += 1
            ff3(bc_anova[bd][ch]['C'])
            
        if bc_anova[bd][ch]['A']>0.05 and bc_anova[bd][ch]['B']>0.05 and bc_anova[bd][ch]['C']>0.05:
            plt.annotate('Gameshowell X', xy=(0.05, 0.85), xycoords='axes fraction')
            
        pos =np.mean(data[data['valence']=='Positive']['value'])
        neg = np.mean(data[data['valence']=='Negative']['value'])
        neu = np.mean(data[data['valence']=='Neutral']['value'])
        
        
        if bc_anova[bd][ch]['A']<0.05:
            if count == 1:
                if neg<neu:
                    plt.annotate('Neg<Neu', xy=(0.05, 0.85), xycoords='axes fraction')
                else:
                    plt.annotate('Neu<Neg', xy=(0.05, 0.85), xycoords='axes fraction')
                    
            if count == 2:
                if neg<neu:
                    plt.annotate('Neg<Neu', xy=(0.05, 0.85), xycoords='axes fraction')
                else:
                    plt.annotate('Neu<Neg', xy=(0.05, 0.85), xycoords='axes fraction')
                    
            elif count == 3:
                if pos<neg<neu:
                     plt.annotate('Pos<Neg<Neu', xy=(0.05, 0.85), xycoords='axes fraction')
                elif pos<neu<neg:
                    plt.annotate('Pos<Neu<Neg', xy=(0.05, 0.85), xycoords='axes fraction')
                elif neg<neu<pos:
                    plt.annotate('Neg<Neu<Pos', xy=(0.05, 0.85), xycoords='axes fraction')
                elif neg<pos<neu:
                    plt.annotate('Neg<Pos<Neu', xy=(0.05, 0.85), xycoords='axes fraction')
                elif neu<pos<neg:
                    plt.annotate('Neu<Pos<Neg', xy=(0.05, 0.85), xycoords='axes fraction')
                elif neu<neg<pos:
                    plt.annotate('Neu<Neg<Pos', xy=(0.05, 0.85), xycoords='axes fraction')
                
                
        if bc_anova[bd][ch]['B']<0.05:
            if count == 1:
                if pos<neg:
                    plt.annotate('Pos<Neg', xy=(0.05, 0.85), xycoords='axes fraction')
                else:
                    plt.annotate('Neg<Pos', xy=(0.05, 0.85), xycoords='axes fraction')
            if count == 2:
                if pos<neg:
                    plt.annotate('Pos<Neg', xy=(0.05, 0.78), xycoords='axes fraction')
                else:
                    plt.annotate('Neg<Pos', xy=(0.05, 0.78), xycoords='axes fraction')
        
        
        if bc_anova[bd][ch]['C']<0.05:
            if count == 1:
                if pos<neu:
                    plt.annotate('Pos<Neu', xy=(0.05, 0.85), xycoords='axes fraction')
                else:
                    plt.annotate('Neu<Pos', xy=(0.05, 0.85), xycoords='axes fraction')
                    
                    
            if count == 2:
                if pos<neu:
                    plt.annotate('Pos<Neu', xy=(0.05, 0.78), xycoords='axes fraction')
                else:
                    plt.annotate('Neu<Pos', xy=(0.05, 0.78), xycoords='axes fraction')
                
        plt.title("BC {} {}".format(bd,ch), pad=20)
        sub += 1

plt.savefig("Fig/COH_BC.jpg", dpi= 600,bbox_inches='tight')


Extracting parameters from C:\Users\user\Desktop\Code\VREED_code\VREED_CONNECTIVITY\VREED_0607_organize\0607VREED정리\..\EEG\02wxw_epoched.set...
Not setting metadata
111 matching events found
No baseline correction applied
0 projection items activated
Ready.
Applying baseline correction (mode: mean)
Extracting parameters from C:\Users\user\Desktop\Code\VREED_code\VREED_CONNECTIVITY\VREED_0607_organize\0607VREED정리\..\EEG\02wxw_epoched.set...
Not setting metadata
111 matching events found
No baseline correction applied
0 projection items activated
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Applying baseline correction (mode: mean)
Extracting parameters from C:\Users\user\Desktop\Code\VREED_code\VREED_CONNECTIVITY\VREED_0607_organize\0607VREED정리\..\EEG\03sf_epoched.set...
Not setting metadata
108 matching events found
No baseline correction applied
0 projection items activated
Ready.
Applying baseline correction (mo

C:\Users\user\AppData\Local\Temp\ipykernel_24188\3822581310.py:21: RuntimeWarning: Estimated head radius (0.0 cm) is below the 3rd percentile for infant head size. Check if the montage_units argument is correct (the default is "mm", but your channel positions may be in different units).
  epochs=mne.io.read_epochs_eeglab(st).apply_baseline(baseline=(-1,0))
C:\Users\user\AppData\Local\Temp\ipykernel_24188\3822581310.py:21: RuntimeWarning: Data file name in EEG.data (15yyw_filter_ICA_pruned_no1699.fdt) is incorrect, the file name must have changed on disk, using the correct file name (15yyw_epoched.fdt).
  epochs=mne.io.read_epochs_eeglab(st).apply_baseline(baseline=(-1,0))


No baseline correction applied
0 projection items activated
Ready.
Applying baseline correction (mode: mean)
Extracting parameters from C:\Users\user\Desktop\Code\VREED_code\VREED_CONNECTIVITY\VREED_0607_organize\0607VREED정리\..\EEG\15yyw_epoched.set...
Not setting metadata
95 matching events found


C:\Users\user\AppData\Local\Temp\ipykernel_24188\3822581310.py:22: RuntimeWarning: Estimated head radius (0.0 cm) is below the 3rd percentile for infant head size. Check if the montage_units argument is correct (the default is "mm", but your channel positions may be in different units).
  epochs=mne.io.read_epochs_eeglab(st).set_eeg_reference('average').apply_baseline(baseline=(-0.5,0))
C:\Users\user\AppData\Local\Temp\ipykernel_24188\3822581310.py:22: RuntimeWarning: Data file name in EEG.data (15yyw_filter_ICA_pruned_no1699.fdt) is incorrect, the file name must have changed on disk, using the correct file name (15yyw_epoched.fdt).
  epochs=mne.io.read_epochs_eeglab(st).set_eeg_reference('average').apply_baseline(baseline=(-0.5,0))


No baseline correction applied
0 projection items activated
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Applying baseline correction (mode: mean)
Extracting parameters from C:\Users\user\Desktop\Code\VREED_code\VREED_CONNECTIVITY\VREED_0607_organize\0607VREED정리\..\EEG\17sh_epoched.set...
Not setting metadata
101 matching events found
No baseline correction applied
0 projection items activated
Ready.
Applying baseline correction (mode: mean)
Extracting parameters from C:\Users\user\Desktop\Code\VREED_code\VREED_CONNECTIVITY\VREED_0607_organize\0607VREED정리\..\EEG\17sh_epoched.set...
Not setting metadata
101 matching events found
No baseline correction applied
0 projection items activated
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Applying baseline correction (mode: mean)
Extracting parameters from C:\Users\user\Desktop\Code\VREED_code\VREED_

In [8]:
globals()['positive_{}_EEG'.format(fr[2])].shape

(708, 59, 59)